In [8]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
import math
import numpy as np
import json
from copy import deepcopy
from config import filename
print(filename)

data/xsmallComplex.json


In [17]:
def n2letter(n):
    '''0 to 'a', 1 to 'b', ... '''
    return chr(96+n)

def string2duration(string):
    ''' "01:50:19.3177493" to duration in seconds'''
    return 3600*int(string[:2]) + 60*int(string[3:5]) + float(string[6:])

In [18]:
def read_data(path):
    global task_count
    global data
    global tasks
    file = open(path)
    data = json.load(file)
    tasks = data['nodes']
    for task in tasks:
        tasks[task]['Data'] = string2duration(tasks[task]['Data'])
    task_count = len(tasks)
    print("Data loaded successfully. Number of tasks: " + str(task_count))

n_cores = 2
read_data(filename)
tasks

Data loaded successfully. Number of tasks: 1181


{'238': {'Data': 6734.5996286, 'Dependencies': [471]},
 '239': {'Data': 3249.8099158, 'Dependencies': [238]},
 '240': {'Data': 1171.3043244, 'Dependencies': [239]},
 '241': {'Data': 4239.4621548, 'Dependencies': [240]},
 '242': {'Data': 5310.3769313, 'Dependencies': [241]},
 '243': {'Data': 7143.7612337, 'Dependencies': [276]},
 '244': {'Data': 5219.794641, 'Dependencies': [243]},
 '245': {'Data': 3585.9751418, 'Dependencies': [244]},
 '246': {'Data': 3860.9575985, 'Dependencies': [245]},
 '247': {'Data': 6140.7610412, 'Dependencies': [246]},
 '248': {'Data': 825.3412346, 'Dependencies': [419]},
 '249': {'Data': 6882.5670595, 'Dependencies': [248]},
 '250': {'Data': 7698.5971496, 'Dependencies': [249]},
 '251': {'Data': 5715.727097, 'Dependencies': [250]},
 '252': {'Data': 2370.4055592, 'Dependencies': [251]},
 '253': {'Data': 746.49309, 'Dependencies': [261]},
 '254': {'Data': 4021.7224612, 'Dependencies': [253]},
 '255': {'Data': 3366.6670991, 'Dependencies': [254]},
 '256': {'Data':

In [19]:
#Tasks to child tasks / Tasks to parents / Task is terminal / Task is inital
task2childs = {task : list() for task in tasks}
task2parents = {task : list() for task in tasks}
for task, info in tasks.items():
    #Add childs
    list_task_parents = info['Dependencies']
    for task_parent in list_task_parents:
        task2childs[str(task_parent)].append(task)
    #Add parents
    task2parents[str(task)] = tasks[task]['Dependencies']
    
def task_is_terminal(task: int):
    return len(task2childs[str(task)]) == 0
def task_is_inital(task: int):
    return len(task2parents[str(task)]) == 0

# print(task2childs)
# print(task2parents)

In [20]:
task2sbl = {}

def save_static_bottom_level(task : int):
    task_duration = tasks[str(task)]["Data"]
    if task_is_terminal(task):
        sbl = task_duration
    else:
        list_sbl_child = list()
        for task_child in task2childs[str(task)]:
            if task_child in task2sbl:
                sbl_child = task2sbl[task_child]
            else:
                sbl_child = save_static_bottom_level(task_child)
            list_sbl_child.append(sbl_child)
        sbl = max(list_sbl_child) + task_duration
                
    task2sbl[task] = sbl
    return sbl

for task in tasks:
    if task_is_inital(task):
        save_static_bottom_level(task)
        
task2sbl



{'507': 3667.5144329,
 '506': 6681.080530499999,
 '505': 6909.478781999999,
 '504': 11108.9015021,
 '510': 15158.0662602,
 '509': 15470.6489142,
 '508': 18130.2235268,
 '515': 22188.0371677,
 '514': 26592.014807300002,
 '513': 32117.182784800003,
 '512': 37113.1559473,
 '511': 43153.0322966,
 '948': 5898.9718898,
 '945': 12145.0742651,
 '942': 17418.0863081,
 '939': 20158.542321999998,
 '937': 26452.042569699996,
 '962': 33334.874556799994,
 '950': 41114.396716999996,
 '946': 45346.5961769,
 '944': 50824.8153649,
 '943': 54537.8210434,
 '941': 60487.831225,
 '940': 62646.0546525,
 '938': 68939.5550794,
 '936': 71586.0971321,
 '935': 76548.75082409999,
 '538': 80368.5573585,
 '537': 83153.4734282,
 '536': 87801.3147652,
 '535': 89764.2615952,
 '534': 93454.9839383,
 '543': 94637.9097422,
 '542': 98312.8769425,
 '541': 99302.071001,
 '540': 106362.1574652,
 '539': 109311.312515,
 '533': 115321.2152544,
 '532': 118852.1466834,
 '531': 119792.14270340001,
 '530': 127616.97866010001,
 '529'

In [21]:
class Node():

    def __init__(self, parent = None, task_to_add = None, core_where_to_add = None, time_task_start = None, graph = None):
        '''Create a Node object ie a partial scheduling
        parent = parent Node, None if root
        task_to_add : task added to the partial schedule
        core_where_to_add : core where to do task
        time_task_start : instant where the core will start computing the task
        '''
        
        if parent is None:
            self.tasks = graph
            self.schedule = {core : list() for core in range(n_cores)}              #lists of the tasks made on each core, ordered by most recent tasks
            self.cores_available_time = {core : 0 for core in range(n_cores)}       #instant t where core will be available
            self.tasks_done_time = {}                                               #instant t where task was done for each task done
            self.tasks_started_time = {}                                            #instant t where task was started for each task done
                        
        else:
            self.tasks = parent.tasks
            
            self.schedule = deepcopy(parent.schedule)
            self.schedule[core_where_to_add].append(task_to_add)
            
            duration_task = self.tasks[task_to_add]['Data']
            self.cores_available_time = deepcopy(parent.cores_available_time)
            self.cores_available_time[core_where_to_add] = time_task_start + duration_task
            
            self.tasks_started_time = deepcopy(parent.tasks_started_time)
            self.tasks_started_time[task_to_add] = time_task_start 
            
            self.tasks_done_time = deepcopy(parent.tasks_done_time)
            self.tasks_done_time[task_to_add] = time_task_start + duration_task            
    
    def __repr__(self):
        string = ''
        for core_schedule in self.schedule.values():
            for task in core_schedule:
                string += n2letter(int(task))
            string += '|'
        return f"{string[:-1]} --- Cores available at time: {self.cores_available_time}. Tasks started times : {self.tasks_started_time}. Tasks done times : {self.tasks_done_time}"
            
    def is_terminal(self):
        '''Return whether a node is a full schedule'''
        return len(self.tasks_done_time) == task_count
    
    def succ(self):                     
        '''Create and return list of child node of self'''
        childs = list()
        
        #On regarde toutes les tâches qu'on va tenter de rajouter
        for task, info in self.tasks.items():
            if task in self.tasks_done_time: #On passe les taches déjà ajoutées
                continue
            if not all([str(task_required) in self.tasks_done_time for task_required in info['Dependencies']]):   #On ne garde que les taches dont toutes les dépendances ont été réalisées
                continue
            
            time_all_dependencies_done = max([0] + [self.tasks_done_time[str(task_required)] for task_required in self.tasks[task]['Dependencies']])
            
            for core in range(n_cores):
                time_core_available = self.cores_available_time[core]
                time_task_start = max(time_core_available, time_all_dependencies_done)  #We wait that the core end its tasks AND that every dependencies was done.
                child = Node(parent = self, task_to_add=task, core_where_to_add=core, time_task_start=time_task_start)
                childs.append(child)
                
        return childs
        
    def cost(self, child_node):
        '''Return the cost of going from self to child_node, a child node of self
        '''
        g_self = max(self.cores_available_time.values())
        g_child = max(child_node.cores_available_time.values())
        cost = g_child - g_self
        return cost
    
    def compute_g(self):
        return max(self.cores_available_time.values())
    
root = Node(graph = tasks)
a = root.succ()[0]
a.succ()

[ϷϿ| --- Cores available at time: {0: 6392.4643117, 1: 0}. Tasks started times : {'919': 0, '927': 3838.0837369}. Tasks done times : {'919': 3838.0837369, '927': 6392.4643117},
 Ϸ|Ͽ --- Cores available at time: {0: 3838.0837369, 1: 6392.4643117}. Tasks started times : {'919': 0, '927': 3838.0837369}. Tasks done times : {'919': 3838.0837369, '927': 6392.4643117},
 Ϸؤ| --- Cores available at time: {0: 9174.205659700001, 1: 0}. Tasks started times : {'919': 0, '1476': 3838.0837369}. Tasks done times : {'919': 3838.0837369, '1476': 9174.205659700001},
 Ϸ|ؤ --- Cores available at time: {0: 3838.0837369, 1: 9174.205659700001}. Tasks started times : {'919': 0, '1476': 3838.0837369}. Tasks done times : {'919': 3838.0837369, '1476': 9174.205659700001},
 Ϸت| --- Cores available at time: {0: 4282.3149335, 1: 0}. Tasks started times : {'919': 0, '1482': 3838.0837369}. Tasks done times : {'919': 3838.0837369, '1482': 4282.3149335},
 Ϸ|ت --- Cores available at time: {0: 3838.0837369, 1: 4282.3149335

In [22]:
def h(node):
    '''Estimated remaining time of the node-schedule for reaching a terminal node.
    Must understimate true value.'''
    successor_tasks = list()
    for task, info in tasks.items():
        if task in node.tasks_done_time: #On passe les taches déjà ajoutées
            continue
        if not all([str(task_required) in node.tasks_done_time for task_required in info['Dependencies']]):   #On ne garde que les taches dont toutes les dépendances ont été réalisées
            continue
        successor_tasks.append(task)
    if successor_tasks == []:
        return 0
    return max([task2sbl[task] for task in successor_tasks])

h(root), h(a)

(1435770.9733583005, 1431932.8896214005)

In [23]:


class A_star():
    def __init__(self, root):
        self.root = root
    
    def find_best_path(self):
        root.g = 0
        root.f = root.g + h(root)
        self.closed_list = []
        self.open_list = [root]
        while len(self.open_list) > 0:
            next_node_to_explore = self.open_list.pop(0)
            self.closed_list.append(next_node_to_explore)
            if next_node_to_explore.is_terminal():
                print("Result:", next_node_to_explore)
                return 
            for node_child in next_node_to_explore.succ():
                # node_child.g = next_node_to_explore.g + next_node_to_explore.cost(node_child)
                node_child.g = node_child.compute_g() #Other method, doesnt work on all Node classes
                node_child.f = node_child.g + h(node_child)
                self.open_list.append(node_child)
            self.open_list.sort(key = lambda node : node.f)
            
        raise Exception("No path from root to a terminal node")

root = Node(graph=tasks)
a_star = A_star(root = root)
a_star.find_best_path()

KeyboardInterrupt: 